In [1]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# local imports
from src.fetch.google import get_place_reviews, get_place_reviews_de, get_place_id, get_rest_info
from src.nlp.dish_extractor import extract_dishes
from src.normalise.normaliser import normalise_review
from src.ranking.functions import assign_rankings   

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
class Restaurant():
    def __init__(self, place_id):
        self.id = place_id
        
        #fetching restaurant info
        info = get_rest_info(place_id)
        self.name = info.get("name")
        self.address = info.get("address")
        self.id_check = info.get("id")
        
        #check if fetch worked
        if str(place_id).strip() != str(self.id_check).strip():
            raise ValueError(f"Restaurant ID mismatch: {place_id} vs {self.id_check}")
        
        #putting in placeholders for important later info
        self.reviews = []
        self.google_url = ""
        self.homepage_url = ""
        self.menu_url = ""

In [3]:
r1_id = get_place_id("funkyfish berlin")
r2_id = get_place_id("standard pizza west berlin")
r3_id = get_place_id("borschert berlin")

r1_id 

--- RESTAURANT DETAILS ---
FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ChIJU7-KvONQqEcRCmPCCvo-Fg4
--- END ---
 
--- RESTAURANT DETAILS ---
Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ChIJc2uwDI1RqEcRfAsy3YxyfIY
--- END ---
 
--- RESTAURANT DETAILS ---
Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ChIJVfEmINtRqEcRsch0pslaanQ
--- END ---
 


'ChIJU7-KvONQqEcRCmPCCvo-Fg4'

In [4]:
r1_reviews = get_place_reviews(r1_id)
r2_reviews = get_place_reviews(r2_id)
r3_reviews = get_place_reviews(r3_id)

# German reviews (don't work well with T5)
r1_de_reviews = get_place_reviews_de(r1_id)
r3_de_reviews = get_place_reviews_de(r3_id)

✅ Google reviews retrieved, found 5 reviews for FunkyFisch
 
✅ Google reviews retrieved, found 5 reviews for Standard Pizza West
 
✅ Google reviews retrieved, found 5 reviews for Borchardt
 


In [5]:
r1_reviews

[{'id': None,
  'source': 'google',
  'author': 'S. K.',
  'text': 'Don’t get distracted by the name :)\nIt’s a very cool, casual, but also a touch of a fancy place!\nThe display of fish looks really nice, and fresh! The price setting feels a little bit higher than excpected, but overall I would recommend this place and be a regular here.',
  'dishes': None,
  'google_rating': None,
  'date': 1757272895,
  'url': 'https://www.google.com/maps/contrib/104531257652878694334/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Dasha Work',
  'text': 'It was an amazing experience! The service in this restaurant is the best from what I saw in Germany so far.\n\nThey proposed me to have an extra meal for free because I didn’t enjoy a soup (not because it was bad, but because it wasn’t for my taste). I said that I’ll take 2 oysters instead, and the waiter said he will bring 3 :) when he came back, he brought 4 oysters for me for free… I was amazed.\n\nAlso the waiter

In [6]:
extract_dishes(r3_reviews, True)

📝 We made a reservation at this German restaurant for lunch. The service was okay, but our main dishes were left out initially.

The tartar and schnitzel were delicious, and the portions were generous. However, the seasonal menu of the deer was average.
🍽️ ['schnitzel', 'tartar', 'schnitzel']

📝 Borchardt is an exceptional place to celebrate special occasions or have a date. Perfect german food, made of high-quality ingredients, in combination with sweet cocktail and freshly-made bread will gift you one of the best gastronomical experiences of your whole life. Atmosphere is quite fancy, and the food prices are higher than in ordinary places in Berlin, and it is totally justified, given the history of the place and the level of service.
And all I would say — it is definitely worth every spent euro! I would like to visit this place one more time (or ten times more)
Would definitely recommend!
P.s. highly recommend to book a table in advance
🍽️ ["'borchardt'", "'borchardt'", "'"]

📝 Borch

In [7]:
assign_rankings(r3_reviews, True)

Assigned ranking -1 to dish 'schnitzel' in review ID 0
Assigned ranking -1 to dish 'tartar' in review ID 0
Assigned ranking -1 to dish ''' in review ID 1
Assigned ranking -1 to dish ''borchardt'' in review ID 1
Assigned ranking -1 to dish 'wiener schnitzel' in review ID 2
Assigned ranking -1 to dish 'steak tartare' in review ID 2
Assigned ranking -1 to dish 'with cranberries' in review ID 3
Assigned ranking -1 to dish 'hot potato salad' in review ID 3
Assigned ranking -1 to dish 'veal schnitzel' in review ID 3
Assigned ranking -1 to dish 'schnitzel' in review ID 4
Assigned ranking -1 to dish 'cod' in review ID 4
Assigned ranking -1 to dish 'riesling' in review ID 4


In [8]:
r3_reviews

[{'id': 0,
  'source': 'google',
  'author': 'Y T',
  'text': 'We made a reservation at this German restaurant for lunch. The service was okay, but our main dishes were left out initially.\n\nThe tartar and schnitzel were delicious, and the portions were generous. However, the seasonal menu of the deer was average.',
  'dishes': [{'name': 'schnitzel', 'ranking': -1},
   {'name': 'tartar', 'ranking': -1}],
  'google_rating': None,
  'date': 1760791579,
  'url': 'https://www.google.com/maps/contrib/109004833449592474577/reviews',
  'language': 'en'},
 {'id': 1,
  'source': 'google',
  'author': 'Олександра Карасьова',
  'text': 'Borchardt is an exceptional place to celebrate special occasions or have a date. Perfect german food, made of high-quality ingredients, in combination with sweet cocktail and freshly-made bread will gift you one of the best gastronomical experiences of your whole life. Atmosphere is quite fancy, and the food prices are higher than in ordinary places in Berlin, an

In [9]:
def flatten_and_sort_dishes(reviews):
    """
    Extracts all dishes from a list of reviews and returns
    them as a flat, sorted list of dicts with metadata.
    """
    dish_entries = []

    for review in reviews:
        source = review.get("source")
        author = review.get("author")
        url = review.get("url")
        date = review.get("date")

        for dish in review.get("dishes", []):
            dish_entries.append({
                "name": dish.get("name"),
                "ranking": dish.get("ranking"),
                "author": author,
                "url": url,
                "date": date,
                "source": source
            })

    # Sort by ranking (None values last)
    dish_entries.sort(key=lambda d: (d["ranking"] is None, d["ranking"]))
    return dish_entries

flatten_and_sort_dishes(r3_reviews)

[{'name': 'schnitzel',
  'ranking': -1,
  'author': 'Y T',
  'url': 'https://www.google.com/maps/contrib/109004833449592474577/reviews',
  'date': 1760791579,
  'source': 'google'},
 {'name': 'tartar',
  'ranking': -1,
  'author': 'Y T',
  'url': 'https://www.google.com/maps/contrib/109004833449592474577/reviews',
  'date': 1760791579,
  'source': 'google'},
 {'name': "'",
  'ranking': -1,
  'author': 'Олександра Карасьова',
  'url': 'https://www.google.com/maps/contrib/103026062957225795450/reviews',
  'date': 1760206989,
  'source': 'google'},
 {'name': "'borchardt'",
  'ranking': -1,
  'author': 'Олександра Карасьова',
  'url': 'https://www.google.com/maps/contrib/103026062957225795450/reviews',
  'date': 1760206989,
  'source': 'google'},
 {'name': 'wiener schnitzel',
  'ranking': -1,
  'author': 'SEZER BULUT',
  'url': 'https://www.google.com/maps/contrib/110446587101289100151/reviews',
  'date': 1759520444,
  'source': 'google'},
 {'name': 'steak tartare',
  'ranking': -1,
  'aut